# Name : Eslam Mohamed 
# Machine Learning 1 Project
# Head Pose Estimation 

In [6]:
!pip install mediapipe

In [14]:
import pickle
import mediapipe as mp
import cv2
import glob
import numpy as np
import csv
from math import cos, sin
import os
import scipy.io as sio
import math
warnings.filterwarnings('ignore')


In [9]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [4]:
with open('./models/model_pitch.pkl', 'rb') as f:

    model_pitch = pickle.load(f)
    
with open('./models/model_yaw.pkl', 'rb') as f:
    model_yaw = pickle.load(f)
    
with open('./models/model_roll.pkl', 'rb') as f:
    model_roll = pickle.load(f)

In [34]:
# List of rows for features
cap = cv2.VideoCapture(0)
faceModule = mp.solutions.face_mesh

with faceModule.FaceMesh(static_image_mode=True) as faces:
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        shape = image.shape
        
        # processing the face to extract the landmark points (468 point) for each x,y,z
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                           )
        
        
        if results.multi_face_landmarks != None: 
      
            for face in results.multi_face_landmarks:


                # Getting two points for normalization
                x_nose, x_corner = face.landmark[4].x , face.landmark[10].x 
                y_nose, y_corner = face.landmark[4].y , face.landmark[10].y

                # un normalize the points
                relative_x_nose, relative_x_corner = int(x_nose * shape[1]), int(x_corner * shape[1])
                relative_y_nose , relative_y_corner = int(y_nose * shape[0]), int(y_corner * shape[0])
              
              # array to calculate the distance
                array_nose = np.array((relative_x_nose,relative_y_nose))
                array_corner = np.array((relative_x_corner,relative_y_corner))

                distance = np.linalg.norm(array_nose - array_corner)
              
                row = list(np.array([[ ((landmark.x *shape[1])-relative_x_nose)/distance, 
                                            ((landmark.y*shape[0])-relative_y_nose)/distance] for landmark in face.landmark]).flatten())
                array_row = np.array((row)).reshape(1,-1)
                pitch = model_pitch.predict(array_row)
                yaw = model_yaw.predict(array_row)
                roll = model_roll.predict(array_row)
                
                yaw = -yaw
                tdx = relative_x_nose
                tdy = relative_y_nose
                size=100
                
                # X-Axis pointing to right. drawn in red
                x1 = size * (cos(yaw) * cos(roll)) + tdx
                y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

                # Y-Axis | drawn in green
                x2 = size * (-cos(yaw) * sin(roll)) + tdx
                y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

                x3 = size * (sin(yaw)) + tdx
                y3 = size * (-cos(yaw) * sin(pitch)) + tdy
                cv2.line(image, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
                cv2.line(image, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
                cv2.line(image, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)
                cv2.imshow('Raw Webcam Feed', image)

            

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 